<a href="https://colab.research.google.com/github/anuj0456/courses/blob/master/competations/he_holiday_season_ensemble_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # !pip install tf-nightly
# !pip uninstall tensorflow  
# !pip install tensorflow

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os
import csv
import sys
import cv2
import time
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from sklearn import preprocessing
from keras.models import Sequential
from tensorflow.keras import layers
from keras.optimizers import Adam, SGD
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import Xception, InceptionResNetV2, EfficientNetB4, ResNet50, EfficientNetB2, EfficientNetB3

In [4]:
# !unzip test.zip
# !unzip train.zip

In [5]:
IMG_SIZE = 224
batch_size = 32
epoch = 10
splits = 5
TRAIN_PATH = '/content/train/'
TEST_PATH = '/content/test/'
csv_path = '/content/train.csv'

In [6]:
df = pd.read_csv(csv_path)

In [7]:
kfold = KFold(splits, True)
# enumerate splits

In [8]:
train_img = []
x = 0
for img_name in tqdm(df['Image'].values):
    img = cv2.imread(TRAIN_PATH + img_name)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    train_img.append(img)
    
x_train = np.array(train_img, np.float32) / 255
print(x_train.shape)

100%|██████████| 6469/6469 [00:02<00:00, 2401.15it/s]


(6469, 224, 224, 3)


In [9]:
label_list = df['Class'].tolist()
label_numeric = {k: v for v, k in enumerate(set(label_list))}
y_train = [label_numeric[k] for k in label_list]
y_train = np.array(y_train)

In [10]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
print(y_train.shape)

(6469, 6)


In [11]:
train_gen = ImageDataGenerator(rotation_range=30, 
                                width_shift_range=0.1, 
                                height_shift_range=0.1,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                )
train_gen.fit(x_train)

In [12]:
rms = RMSprop(lr = 0.001, momentum=0.9, clipnorm = 1., clipvalue = 1.)
adam = Adam(lr = 1e-4)
sgd = SGD(lr = 1e-4, momentum=0.9)

In [13]:
def get_model1():
    core = EfficientNetB4(include_top=False, input_tensor=None,weights='imagenet',input_shape=(IMG_SIZE, IMG_SIZE,3))
    model = Sequential()
    model.add(core)
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    
    return model

In [14]:
model1 = get_model1()
model1.compile(optimizer= adam, metrics=['accuracy'], loss='categorical_crossentropy')

In [15]:
model_save1 = ModelCheckpoint("/content/model_1", 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'loss', 
                             mode = 'min', verbose = 1)
early_stop1 = EarlyStopping(monitor = 'loss', min_delta = 0.01, 
                           patience = 2, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr1 = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, 
                              patience = 2, min_delta = 0.01, 
                              mode = 'min', verbose = 1)

In [16]:
# for train, test in kfold.split(x_train):
history1 = model1.fit(train_gen.flow(x_train, y_train, batch_size=batch_size), epochs=epoch,callbacks=[model_save1])

Epoch 1/10
203/203 [==============================] - 121s 498ms/step - loss: 1.4348 - accuracy: 0.5136

Epoch 00001: loss improved from inf to 1.09392, saving model to /content/model_1
Epoch 2/10
203/203 [==============================] - 101s 498ms/step - loss: 0.6808 - accuracy: 0.7497

Epoch 00002: loss improved from 1.09392 to 0.65293, saving model to /content/model_1
Epoch 3/10
203/203 [==============================] - 101s 494ms/step - loss: 0.5288 - accuracy: 0.8169

Epoch 00003: loss improved from 0.65293 to 0.51319, saving model to /content/model_1
Epoch 4/10
203/203 [==============================] - 101s 497ms/step - loss: 0.4132 - accuracy: 0.8528

Epoch 00004: loss improved from 0.51319 to 0.40796, saving model to /content/model_1
Epoch 5/10
203/203 [==============================] - 101s 498ms/step - loss: 0.3372 - accuracy: 0.8844

Epoch 00005: loss improved from 0.40796 to 0.35452, saving model to /content/model_1
Epoch 6/10
203/203 [==============================] - 

In [17]:
def get_model2():
    core = InceptionResNetV2(include_top=False, input_tensor=None,weights='imagenet',input_shape=(IMG_SIZE, IMG_SIZE,3))
    model = Sequential()
    model.add(core)
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    
    return model

In [18]:
model2 = get_model2()
model2.compile(optimizer= adam, metrics=['accuracy'], loss='categorical_crossentropy')

In [19]:
model_save2 = ModelCheckpoint("/content/model_2", 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'loss', 
                             mode = 'min', verbose = 1)
early_stop2 = EarlyStopping(monitor = 'loss', min_delta = 0.01, 
                           patience = 2, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr2 = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, 
                              patience = 2, min_delta = 0.01, 
                              mode = 'min', verbose = 1)

In [20]:
history2 = model2.fit(train_gen.flow(x_train, y_train, batch_size=batch_size), epochs=epoch, callbacks=[model_save2])

Epoch 1/10
203/203 [==============================] - 94s 360ms/step - loss: 1.4512 - accuracy: 0.5267

Epoch 00001: loss improved from inf to 1.07473, saving model to /content/model_2
Epoch 2/10
203/203 [==============================] - 74s 361ms/step - loss: 0.6327 - accuracy: 0.7885

Epoch 00002: loss improved from 1.07473 to 0.60552, saving model to /content/model_2
Epoch 3/10
203/203 [==============================] - 74s 361ms/step - loss: 0.4651 - accuracy: 0.8423

Epoch 00003: loss improved from 0.60552 to 0.47636, saving model to /content/model_2
Epoch 4/10
203/203 [==============================] - 73s 360ms/step - loss: 0.3808 - accuracy: 0.8737

Epoch 00004: loss improved from 0.47636 to 0.38969, saving model to /content/model_2
Epoch 5/10
203/203 [==============================] - 74s 362ms/step - loss: 0.2994 - accuracy: 0.9008

Epoch 00005: loss improved from 0.38969 to 0.31673, saving model to /content/model_2
Epoch 6/10
203/203 [==============================] - 74s 3

In [21]:
def get_model3():
    core = EfficientNetB2(include_top=False, input_tensor=None,weights='imagenet',input_shape=(IMG_SIZE, IMG_SIZE,3))
    model = Sequential()
    model.add(core)
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    
    return model

In [22]:
model3 = get_model3()
model3.compile(optimizer= adam, metrics=['accuracy'], loss='categorical_crossentropy')

31793152/31790344 [==============================] - 1s 0us/step


In [23]:
model_save3 = ModelCheckpoint("/content/model_3", 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'loss', 
                             mode = 'min', verbose = 1)
early_stop3 = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr3 = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.001, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

In [24]:
history3 = model3.fit( train_gen.flow(x_train, y_train, batch_size=batch_size), epochs=epoch, callbacks=[model_save3])

Epoch 1/10
203/203 [==============================] - 86s 356ms/step - loss: 1.3942 - accuracy: 0.5748

Epoch 00001: loss improved from inf to 1.02699, saving model to /content/model_3
Epoch 2/10
203/203 [==============================] - 73s 356ms/step - loss: 0.5972 - accuracy: 0.7919

Epoch 00002: loss improved from 1.02699 to 0.58809, saving model to /content/model_3
Epoch 3/10
203/203 [==============================] - 72s 354ms/step - loss: 0.4583 - accuracy: 0.8330

Epoch 00003: loss improved from 0.58809 to 0.44288, saving model to /content/model_3
Epoch 4/10
203/203 [==============================] - 72s 355ms/step - loss: 0.3830 - accuracy: 0.8703

Epoch 00004: loss improved from 0.44288 to 0.36518, saving model to /content/model_3
Epoch 5/10
203/203 [==============================] - 73s 357ms/step - loss: 0.3010 - accuracy: 0.8936

Epoch 00005: loss improved from 0.36518 to 0.32096, saving model to /content/model_3
Epoch 6/10
203/203 [==============================] - 73s 3

In [25]:
def get_model4():
    core = EfficientNetB3(include_top=False, input_tensor=None,weights='imagenet',input_shape=(IMG_SIZE, IMG_SIZE,3))
    model = Sequential()
    model.add(core)
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    
    return model

In [26]:
model4 = get_model4()
model4.compile(optimizer= adam, metrics=['accuracy'], loss='categorical_crossentropy')

43941888/43941136 [==============================] - 2s 0us/step


In [27]:
model_save4 = ModelCheckpoint("/content/model_4", 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'loss', 
                             mode = 'min', verbose = 1)
early_stop4 = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr4 = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.001, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

In [28]:
history4 = model4.fit( train_gen.flow(x_train, y_train, batch_size=batch_size), epochs=epoch, callbacks=[model_save4])

Epoch 1/10
203/203 [==============================] - 97s 398ms/step - loss: 1.4459 - accuracy: 0.5826

Epoch 00001: loss improved from inf to 0.99991, saving model to /content/model_4
Epoch 2/10
203/203 [==============================] - 81s 397ms/step - loss: 0.5885 - accuracy: 0.8021

Epoch 00002: loss improved from 0.99991 to 0.57157, saving model to /content/model_4
Epoch 3/10
203/203 [==============================] - 81s 397ms/step - loss: 0.4020 - accuracy: 0.8610

Epoch 00003: loss improved from 0.57157 to 0.40824, saving model to /content/model_4
Epoch 4/10
203/203 [==============================] - 82s 400ms/step - loss: 0.3185 - accuracy: 0.8913

Epoch 00004: loss improved from 0.40824 to 0.31699, saving model to /content/model_4
Epoch 5/10
203/203 [==============================] - 81s 398ms/step - loss: 0.2520 - accuracy: 0.9130

Epoch 00005: loss improved from 0.31699 to 0.25465, saving model to /content/model_4
Epoch 6/10
203/203 [==============================] - 81s 3

In [29]:
try:
  os.remove('/content/submission.csv')
except Exception as e:
  print("No file exists")
with open('/content/submission.csv', 'w', newline='') as file:
      writer = csv.writer(file, delimiter = ',')
      writer.writerow(["Image", "Class"])

In [30]:
members = [model1, model2, model3, model4]

In [31]:
def ensemble_predictions(members, testX):
	# make predictions
	yhats = [model.predict(testX) for model in members]
	yhats = np.array(yhats)
	summed = np.sum(yhats, axis=0)
	return summed

In [32]:
test_images = os.listdir(TEST_PATH)
test_img = []
for img_name in tqdm(test_images):
    img = cv2.imread(TEST_PATH + img_name)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    test_img.append(img)

100%|██████████| 3489/3489 [00:01<00:00, 2761.04it/s]


In [33]:
x_test = np.array(test_img, np.float32) / 255

In [34]:
predictions = ensemble_predictions(members,x_test)

In [35]:
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in label_numeric.items()}
pred_labels = [rev_y[k] for k in predictions]

In [36]:
sub = pd.DataFrame({'Image': test_images, 'Class': pred_labels})
sub.to_csv('/content/submission.csv', index = False)

In [37]:
from google.colab import files
files.download('/content/submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
import gc 
gc.collect()

2804